### 1. Import Library

In [141]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import re

# Mengatur agar semua kolom ditampilkan
pd.set_option('display.max_columns', 100)

# Mengatur agar semua baris ditampilkan
pd.set_option('display.max_rows', 100)

# Mengatur lebar kolom maksimum agar teks tidak terpotong
pd.set_option('display.max_colwidth', 100)

# Mengatur lebar tampilan untuk menghindari pembungkusan baris
pd.set_option('display.width', 100)

### 2. Function

In [142]:
# Fungsi untuk membersihkan string dan mengkonversinya menjadi float
def clean_and_convert(value):
    # Cek apakah nilai adalah string, jika tidak, langsung kembalikan
    if not isinstance(value, str):
        return value
    # Hapus karakter mata uang dan unit
    cleaned_value = re.sub(r'[^\d.]', '', value)
    # Jika hasil pembersihan adalah string kosong, kembalikan NaN
    if cleaned_value == '':
        return float('nan')
    # Konversi ke float
    return float(cleaned_value)

### 3. Load Data

In [143]:
# Muat data
df = pd.read_csv('house_listings.csv')

### 3. Normalisasi

#### 3.1 Bersihkan dan Konversi Kolom

In [144]:
# Bersihkan dan konversi kolom 'Price' dan 'Price_1m2'
df['price'] = df['price'].apply(lambda x: clean_and_convert(str(x)))
df['price_1m2'] = df['price_1m2'].apply(lambda x: clean_and_convert(str(x)))

In [145]:
# Ganti NaN dengan nilai yang diinginkan, misalnya 0 atau nilai rata-rata
df.fillna(0, inplace=True)

#### 3.2 Lakukan Normalisasi

In [146]:
# Pilih kolom untuk normalisasi
features_to_normalize = ['price', 'price_1m2']

# Lakukan normalisasi
scaler = MinMaxScaler()
df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])
# Tampilkan data
print(features_to_normalize)

['price', 'price_1m2']


### 4. Enconding

In [147]:
# Lakukan one-hot encoding
df = pd.get_dummies(df, columns=['category', 'title_deed', 'repair', 'mortgage'])

### 5. Agregasi

In [148]:
# Tambahkan kolom baru 'building_type' berdasarkan kondisi kolom boolean
df['building_type'] = df.apply(lambda row: 'Köhnə tikili' if row['category_Köhnə tikili'] else ('Yeni tikili' if row['category_Yeni tikili'] else 'Unknown'), axis=1)

# Sekarang Anda dapat melakukan agregasi data menggunakan kolom 'building_type'
aggregated_data = df.groupby(['room_number', 'building_type']).agg({
    'price': ['mean', 'median', lambda x: x.mode().iloc[0] if not x.empty else float('nan')]
}).reset_index()

# Mengganti nama kolom hasil agregasi
aggregated_data.columns = ['room_number', 'building_type', 'mean_price', 'median_price', 'modus_price']

# Tampilkan data agregasi
print(aggregated_data)

    room_number building_type  mean_price  median_price  modus_price
0           0.0       Unknown    0.000000      0.000000     0.000000
1           1.0  Köhnə tikili    0.017911      0.017255     0.018627
2           1.0   Yeni tikili    0.023961      0.023529     0.025490
3           2.0  Köhnə tikili    0.023535      0.022549     0.023529
4           2.0   Yeni tikili    0.032188      0.030392     0.028431
5           3.0  Köhnə tikili    0.032684      0.030098     0.034314
6           3.0   Yeni tikili    0.050076      0.048039     0.049020
7           4.0  Köhnə tikili    0.041505      0.035294     0.032353
8           4.0   Yeni tikili    0.079519      0.072549     0.068627
9           5.0  Köhnə tikili    0.054232      0.039510     0.043137
10          5.0   Yeni tikili    0.123101      0.101961     0.088235
11          6.0  Köhnə tikili    0.122573      0.098039     0.053922
12          6.0   Yeni tikili    0.165133      0.147059     0.196078
13          7.0  Köhnə tikili    0

In [150]:

# print(df.columns)


Index(['price', 'currency', 'price_1m2', 'title', 'address', 'floor', 'area', 'url', 'room_number',
       'category_0', 'category_Köhnə tikili', 'category_Yeni tikili', 'title_deed_0',
       'title_deed_var', 'title_deed_yoxdur', 'repair_0', 'repair_var', 'repair_yoxdur',
       'mortgage_0', 'mortgage_var', 'building_type'],
      dtype='object')
